<a href="https://colab.research.google.com/github/BotenAqua/WarframeInventoryHelper/blob/master/WarframeInventoryHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests

In [24]:
def pageTableExtractor(page, first_row_is_header = True):
  try:
    data = requests.get(page)
  except NameError:
    import requests
    data = requests.get(page)
  except ModuleNotFoundError:
    print("Requests module not found! Aborting...")
    return
  data = data.text
  #print(data.find("<table"), data.find("</table>"))
  data = data[data.find("<table"):data.find("</table>")]

  if first_row_is_header:
    header = []
    row = data[data.find("<tr>")+4:data.find("</tr>")]
    for _ in range(row.count("</th>")):
      do = row[3:].find("<")+2
      #print(row, row.find(">")+1,do)
      header.append(row[row.find(">")+1:do])
      row = row[do+6:]
    print(header)
  return data

pageTableExtractor("https://warframe.fandom.com/wiki/Void_Relic/ByRewards/SimpleTable")


['Item', 'Part', 'Tier', 'Name', 'Rarity', 'Vaulted?']


'<table class="article-table sortable" style="width: 600px;margin: auto;" cellspacing="1" cellpadding="1" border="2">\n\n<tr>\n<th scope="col">Item\n</th><th scope="col">Part\n</th><th scope="col">Tier\n</th><th scope="col">Name\n</th><th scope="col">Rarity\n</th><th scope="col">Vaulted?\n</th></tr>\n<tr>\n<td><a href="/wiki/Akbolto_Prime" title="Akbolto Prime">Akbolto Prime</a></td><td>Barrel</td><td>Lith</td><td><span style="border-bottom: 1px dotted;" class="relic-tooltip" data-param="Lith A3"><a href="/wiki/Lith_A3" title="Lith A3">A3</a></span></td><td>Uncommon</td><td>No\n</td></tr>\n<tr>\n<td><a href="/wiki/Akbolto_Prime" title="Akbolto Prime">Akbolto Prime</a></td><td>Barrel</td><td>Lith</td><td><span style="border-bottom: 1px dotted;" class="relic-tooltip" data-param="Lith C2"><a href="/wiki/Lith_C2" title="Lith C2">C2</a></span></td><td>Uncommon</td><td>Yes\n</td></tr>\n<tr>\n<td><a href="/wiki/Akbolto_Prime" title="Akbolto Prime">Akbolto Prime</a></td><td>Barrel</td><td>Lith